In [1]:
import os
original_dir = os.getcwd()
os.chdir(os.path.dirname(os.path.dirname(os.getcwd())))
exec(open('setup/default.py').read())

In [31]:
raw_df = pd.read_csv(os.getcwd()+'/data/raw_data.csv')

## 식단 
### 게시글 기반

In [32]:
# 필터링 
body_df = raw_df[raw_df.type=='body'].copy()
body_df.loc[:, 'body'] = body_df['body'].str.replace(r'\s+|=', '', regex=True) # body에 \s(공백 = 줄바꿈, 띄어쓰기, 탭) 모두 (+) 제거 
body_df = body_df[body_df['body'].str.contains("[COURSE1]")] # COURSE1이 있는 메뉴만 남김. (런치 스페셜은 예외 케이스로 지워두기)
body_df.loc[:, 'body'] = body_df['body'].str.replace(r'Kcal', 'kcal', regex=True) # 칼로리 다르게 표기 된것 정리 

In [33]:
# 오전/오후를 24시간 형식으로 변환
body_df['time'] = body_df['time'].str.replace('오전', 'AM').str.replace('오후', 'PM')

# time 열을 time 형식으로 변환
body_df['time'] = pd.to_datetime(body_df['time'], format='%Y년 %m월 %d일 %p %I:%M', errors='coerce')

# 일(day), 시간 (hour), 요일(day of week) 추출
body_df['day'] = body_df['time'].dt.date
body_df['hour'] = body_df['time'].dt.strftime('%H:%M') 
body_df['day_of_week'] = body_df['time'].dt.strftime('%A')

# 시간 기반 중식, 석식 구분 
body_df['type'] = np.where(body_df['time'].dt.strftime('%H').astype(int)<=13, 'lunch',
                   np.where(body_df['time'].dt.strftime('%H').astype(int)>=16, 'dinner',''))

In [53]:
# 가끔 점심 포스팅이 누락된 경우도 있긴 함. (아래는 석식은 포스팅이 있는데, 점심은 포스팅이 없는 경우임.)
[x for x in body_df[body_df.type == 'dinner'].day.tolist() if x not in body_df[body_df.type == 'lunch'].day.tolist()]

[datetime.date(2023, 5, 11),
 datetime.date(2023, 6, 8),
 datetime.date(2023, 7, 11),
 datetime.date(2023, 9, 14),
 datetime.date(2023, 10, 12),
 datetime.date(2023, 10, 13),
 datetime.date(2023, 11, 9),
 datetime.date(2023, 12, 14),
 datetime.date(2024, 1, 11),
 datetime.date(2024, 1, 30),
 datetime.date(2022, 12, 12),
 datetime.date(2023, 1, 12),
 datetime.date(2023, 2, 9),
 datetime.date(2023, 3, 9),
 datetime.date(2023, 4, 13)]

In [54]:
# 코스 별 특성
def extract_course_info(text):
    # . : 모든 단일 문자와 일치
    # * : 앞의 문자가 0번 이상 반복
    # ? : 비탐욕적(가능한 한 적은 문자를) 일치 (? 이 있으면 가장 먼저 나온 kcal에서 끊어주기)
    # () : 일치된 부분을 캡처하는 그룹 생성
    course1_pattern = r'\[COURSE1\](.*?)(\d+)kcal\(단백질:(\d+)g\)Na함량:(\d*)mg' # Na 함량은 겂이 비어있는 경우도 있어서 * 표시 
    course2_pattern = r'\[COURSE2\](.*?)(\d+)kcal\(단백질:(\d+)g\)Na함량:(\d*)mg'

    course1_match = re.search(course1_pattern, text)
    course2_match = re.search(course2_pattern, text)

    if course1_match:
        course1_menu = course1_match.group(1)
        course1_kcal = int(course1_match.group(2)) if course1_match.group(2) != "" else np.nan
        course1_protein = int(course1_match.group(3)) if course1_match.group(3) != "" else np.nan
        course1_na = int(course1_match.group(4)) if course1_match.group(4) != "" else np.nan
    else:
        course1_menu, course1_kcal, course1_protein, course1_na = np.nan, np.nan, np.nan, np.nan

    if course2_match:
        course2_menu = course2_match.group(1)
        course2_kcal = int(course2_match.group(2)) if course2_match.group(2) != "" else np.nan
        course2_protein = int(course2_match.group(3)) if course2_match.group(3) != "" else np.nan
        course2_na = int(course2_match.group(4)) if course2_match.group(4) != "" else np.nan
    else:
        course2_menu, course2_kcal, course2_protein, course2_na = np.nan, np.nan, np.nan, np.nan

    return (course1_menu, course1_kcal, course1_protein, course1_na,
            course2_menu, course2_kcal, course2_protein, course2_na)

# 새로운 컬럼 추가
body_df[['course1', 'course1_kcal', 'course1_protein', 'course1_na',
    'course2', 'course2_kcal', 'course2_protein', 'course2_na']] = body_df['body'].apply(lambda x: pd.Series(extract_course_info(x)))

In [55]:
body_df.to_csv(os.getcwd()+'/data/preproc/main/menu.csv', sep=',' ,na_rep='NaN',index = False)

## 품절 여부 
### 댓글 기반

In [44]:
# 관리자가 쓴 댓글만 추출
comment_df = raw_df[(raw_df.type=='comment') & 
                    (raw_df.writer.isin(['Kitchen B1','kitchen B1','B1_Kitchen','원지연','kitchen_B1']))]
comment_df.loc[:, 'body'] = comment_df['body'].str.replace(r'\s+|=', '', regex=True)

# 시간 설정
# 오전/오후를 24시간 형식으로 변환
comment_df.loc[:, 'time'] = comment_df['time'].str.replace('오전', 'AM').str.replace('오후', 'PM')

# [참고] comment_df는 날짜 형식이 2가지로 남음. (ex. 2022년 9월 21일, 4월 18일 오후 1:48 )
format2 = '%m월 %d일 %p %I:%M'
comment_df.loc[:, 'time2'] = pd.to_datetime(comment_df['time'], format=format2, errors='coerce')
# time 열을 time 형식으로 변환
comment_df.loc[:, 'time2']= pd.to_datetime(comment_df['time2'], format='%Y년 %m월 %d일 %p %I:%M', errors='coerce')
# 일(day), 시간 (hour), 요일(day of week) 추출
comment_df.loc[:, 'hour'] = comment_df['time2'].dt.strftime('%H:%M') 
# 불필요한 열 제거
comment_df.drop(columns=['time2'], inplace=True)


/Users/user/first_pytorch/lib/python3.7/site-packages/pandas/core/frame.py:4315: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [45]:
soldout_df= comment_df[comment_df['body'].str.contains("품절되|품절로|품절입")] # 품절되었습니다, 품절로, 품절입니다

In [46]:
# [참고] "품절되|품절로|품절입"는 안들어가지만, 품절이 들어가는 것중에 놓친 부분이 있는지? 
# 없음. -> 품절 대체찬 관련 내용임. 
print(comment_df[(comment_df['body'].str.contains("품절"))&(~comment_df['body'].str.contains("품절되|품절로|품절입"))])

        post_no     type      writer                           body  \
141   926901281  comment  Kitchen B1          2코너품절대체찬두부쑥갓나물이제공됩니다.   
198   926901318  comment  Kitchen B1       1코너품절대체찬으로2코너에콩자반이제공됩니다.   
232   926901334  comment  Kitchen B1  중식1코너품절대체찬으로2코너에서반반콩자반제공드립니다.   
233   926901334  comment  Kitchen B1     2코너품절대체찬반반콩자반->군만두로제공드립니다.   
252   926901350  comment  Kitchen B1  중식2코너품절대체찬으로1코너에서김자반볶음이제공됩니다.   
272   926901363  comment  Kitchen B1    1코너품절대체찬으로2코너에서쥐어채조림이제공됩니다.   
378   926901435  comment  Kitchen B1      1코너품절대체찬으로2코너에서부꾸미가제공됩니다.   
398   926901445  comment  Kitchen B1    2코너품절대체찬으로1코너에서감자샐러드가제공됩니다.   
697   926901641  comment  Kitchen B1     품절추가찬으로1코너에서감자샐러드를제공해드립니다.   
808   926901705  comment  Kitchen B1   중식1코너품절찬으로2코스에서동그랑땡조림이제공됩니다.   
997   926901833  comment  Kitchen B1          1코너품절찬으로순두부샐러드가제공됩니다.   
1010  926901843  comment  Kitchen B1     1코너품절찬으로2코너에서고구마샐러드가제공됩니다.   
1797  926901130  comment  Kitchen B1                품절대체찬계란찜제공드립니다.   
1809  

In [47]:

# 패턴 정의
pattern = r"(1코너|1코스|2코너|2코스)"

# 먼저 나오는 패턴 추출 (동시에 2개가 나오는 경우도 있음. ex. 1코너 품절로, 2코너에 추가찬 제공합니다.)

def classify(text):
    match = re.search(pattern, text)
    if match:
        found = match.group(0) # 더 먼저 나오는 것 기준
        if found in ["1코너", "1코스"]:
            return 1
        elif found in ["2코너", "2코스"]:
            return 2
    else:
        return None
# 데이터프레임에 새로운 열 추가
soldout_df['soldout'] = soldout_df['body'].apply(classify)



/Users/user/first_pytorch/lib/python3.7/site-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [48]:
soldout_df[soldout_df.soldout.isin([1,2])].sort_values(by=['soldout']).to_csv(os.getcwd()+'/data/preproc/main/soldout.csv', sep=',' ,na_rep='NaN',index = False)